In [ ]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(0) 

import pandas as pd
import random
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

import os

import numpy as np
import torch.nn.functional as F

from google.colab import drive
drive.mount('/content/drive')

from sklearn.metrics import accuracy_score
from datetime import datetime
from IPython.display import clear_output 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
#!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

In [ ]:

train_dataset=[]
graphlist=[]
import networkx as nx
import torch_geometric
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader
#from torch.utils.data import DataLoader
from torch_geometric.utils import erdos_renyi_graph, to_networkx, from_networkx
#%cd /content/drive/MyDrive/GNN_DATA/ADNI/ALL_NETWORKS
%cd /content/drive/MyDrive/ALL_NETWORKS

for i in range(1, 110):
    if os.path.isfile('AD_'+'{0:0>3}'.format(i)+'.csv'):
      tmp=pd.read_csv('AD_'+'{0:0>3}'.format(i)+'.csv', header=None)
      tmp=(tmp.to_numpy())
      tmp=torch.from_numpy(tmp)  
      tmp=torch.div(tmp,tmp.max())
      #print([tmp.type(torch.FloatTensor),torch.ones(1).type(torch.long)])
      G = nx.from_numpy_matrix(np.array(tmp))
      strength = G.degree(weight='weight')
      x = []
      for i in strength:
        x.extend([i[1]])
      x = np.array(x).reshape(90,1)
      data = from_networkx(G)
      #x = torch.tensor([[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1] ], dtype=torch.float)
      data.x = torch.FloatTensor(x)
      data.y = torch.tensor([1])
      graphlist.append(data)
      #graphlist.append([tmp.type(torch.FloatTensor),torch.ones(1).type(torch.long)])

for i in range(1, 110):
    if os.path.isfile('CN_'+'{0:0>3}'.format(i)+'.csv'):
      tmp=pd.read_csv('CN_'+'{0:0>3}'.format(i)+'.csv', header=None)
      tmp=(tmp.to_numpy())
      tmp=torch.from_numpy(tmp) 
      tmp=torch.div(tmp, tmp.max())
      #print([tmp.type(torch.FloatTensor),torch.ones(1).type(torch.long)])
      G = nx.from_numpy_matrix(np.array(tmp))
      strength = G.degree(weight='weight')
      x = []
      for i in strength:
        x.extend([i[1]])
      x = np.array(x).reshape(90,1)
      data = from_networkx(G)
      #x = torch.tensor([[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1] ], dtype=torch.float)
      data.x = torch.FloatTensor(x)
      data.y = torch.tensor([0])
      graphlist.append(data)
      #graphlist.append([data, torch.zeros(1).type(torch.long)])

      #graphlist.append([tmp.type(torch.FloatTensor),torch.zeros(1).type(torch.long)])

/content/drive/.shortcut-targets-by-id/10M6M6of9sCS0_CWq5d9EF6LhBCK9klNf/ALL_NETWORKS


In [ ]:
graphlist[0]

Data(edge_index=[2, 3814], weight=[3814], num_nodes=90, x=[90, 1], y=[1])

In [ ]:
%cd /content/drive/MyDrive
pos = pd.read_csv("node_aal90.csv", header = None)

/content/drive/MyDrive


In [ ]:
position1 = []
for i in pos[0]:
  position1.extend([i])
position1 = np.array(position1).reshape(90,1)

position2 = []
for i in pos[1]:
  position2.extend([i])
position2 = np.array(position2).reshape(90,1)

position3 = []
for i in pos[2]:
  position3.extend([i])
position3 = np.array(position3).reshape(90,1)

position = np.concatenate((position1, position2, position3), axis = 1) 

In [ ]:
for i in range(len(graphlist)):
  graphlist[i].x = torch.FloatTensor(np.concatenate((graphlist[i].x, position), axis = 1))

In [ ]:
graphlist[-1].x

tensor([[ 4.0284e+00, -3.8650e+01, -5.6800e+00,  5.0940e+01],
        [ 3.8760e+00,  4.1370e+01, -8.2100e+00,  5.2090e+01],
        [ 1.7597e+00, -1.8450e+01,  3.4810e+01,  4.2200e+01],
        [ 2.4419e+00,  2.1900e+01,  3.1120e+01,  4.3820e+01],
        [ 1.1370e+00, -1.6560e+01,  4.7320e+01, -1.3310e+01],
        [ 1.3979e+00,  1.8490e+01,  4.8100e+01, -1.4020e+01],
        [ 5.3747e+00, -3.3430e+01,  3.2730e+01,  3.5460e+01],
        [ 5.4341e+00,  3.7590e+01,  3.3060e+01,  3.4040e+01],
        [ 6.8992e-01, -3.0650e+01,  5.0430e+01, -9.6200e+00],
        [ 1.0284e+00,  3.3180e+01,  5.2590e+01, -1.0730e+01],
        [ 1.1602e+00, -4.8430e+01,  1.2730e+01,  1.9020e+01],
        [ 1.7028e+00,  5.0200e+01,  1.4980e+01,  2.1410e+01],
        [ 1.9070e+00, -4.5580e+01,  2.9910e+01,  1.3990e+01],
        [ 1.6641e+00,  5.0330e+01,  3.0160e+01,  1.4170e+01],
        [ 1.0749e+00, -3.5980e+01,  3.0710e+01, -1.2110e+01],
        [ 1.7571e+00,  4.1220e+01,  3.2230e+01, -1.1910e+01],
        

In [ ]:
dataset = random.sample(graphlist, len(graphlist))
train_dataset = dataset[:150]
test_dataset = dataset[150:]
train_loader = DataLoader(train_dataset, batch_size=25, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=25, shuffle=False)

In [ ]:
from torch_geometric.nn import GATConv, GraphConv
from torch_geometric.nn.conv.gatv2_conv import GATv2Conv
from torch_geometric.nn import global_mean_pool, global_sort_pool

class GNN(torch.nn.Module):
    def __init__(self):
        super(GNN, self).__init__()

        self.conv1 = GraphConv(in_channels=4, out_channels=16)
        self.conv2 = GraphConv(in_channels=16, out_channels=32)
        self.conv3 = GraphConv(in_channels=32, out_channels=32)
        self.lin1 = torch.nn.Linear(32, 16)
        self.lin2 = torch.nn.Linear(16, 1)
        self.sig = torch.nn.Sigmoid()

    def forward(self, data):
        x, edge_index, weight, batch = data.x, data.edge_index, data.weight, data.batch 
        x = self.conv1(x, edge_index, weight)
        x = F.relu(x)
        #print(x.size())
        x = self.conv2(x, edge_index, weight)
        x = F.relu(x)
        #print(x.size())
        x = self.conv3(x, edge_index, weight)
        x = F.relu(x)
        #print(x.size())
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        #x = F.dropout(x, p=0.2, training=self.training)
        x = self.lin1(x)
        x = self.lin2(x)
        #print(x)
        
        return x
        #return F.log_softmax(x, dim=1)

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNN().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6923
Epoch: 002, Train Acc: 0.5667, Test Acc: 0.5000, Loss: 0.6575
Epoch: 003, Train Acc: 0.5667, Test Acc: 0.5192, Loss: 0.6420
Epoch: 004, Train Acc: 0.6000, Test Acc: 0.5000, Loss: 0.5851
Epoch: 005, Train Acc: 0.6267, Test Acc: 0.5769, Loss: 0.6719
Epoch: 006, Train Acc: 0.6000, Test Acc: 0.5577, Loss: 0.6523
Epoch: 007, Train Acc: 0.6200, Test Acc: 0.4615, Loss: 0.6521
Epoch: 008, Train Acc: 0.6267, Test Acc: 0.5962, Loss: 0.7824
Epoch: 009, Train Acc: 0.5800, Test Acc: 0.5769, Loss: 0.7502
Epoch: 010, Train Acc: 0.6533, Test Acc: 0.5385, Loss: 0.7025
Epoch: 011, Train Acc: 0.6200, Test Acc: 0.5385, Loss: 0.6362
Epoch: 012, Train Acc: 0.6067, Test Acc: 0.6154, Loss: 0.6303
Epoch: 013, Train Acc: 0.6400, Test Acc: 0.5769, Loss: 0.6220
Epoch: 014, Train Acc: 0.6200, Test Acc: 0.5577, Loss: 0.6171
Epoch: 015, Train Acc: 0.5733, Test Acc: 0.4808, Loss: 0.6762
Epoch: 016, Train Acc: 0.6000, Test Acc: 0.5000, Loss: 0.8017
Epoch: 0

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import GCNConv, JumpingKnowledge, global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.num_layers  = 3
        self.hidden = 16
        self.conv1 = GCNConv(4, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(GCNConv(self.hidden, self.hidden))
        self.lin1 = Linear(self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index,weight, batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index, weight))
        x = global_mean_pool(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 500):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.7075
Epoch: 002, Train Acc: 0.4867, Test Acc: 0.5192, Loss: 0.7144
Epoch: 003, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6865
Epoch: 004, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6915
Epoch: 005, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.7082
Epoch: 006, Train Acc: 0.5200, Test Acc: 0.4808, Loss: 0.6854
Epoch: 007, Train Acc: 0.4733, Test Acc: 0.5000, Loss: 0.7010
Epoch: 008, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.7071
Epoch: 009, Train Acc: 0.5200, Test Acc: 0.4808, Loss: 0.6865
Epoch: 010, Train Acc: 0.5267, Test Acc: 0.4615, Loss: 0.6841
Epoch: 011, Train Acc: 0.5667, Test Acc: 0.4615, Loss: 0.6814
Epoch: 012, Train Acc: 0.6400, Test Acc: 0.5192, Loss: 0.6837
Epoch: 013, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.7314
Epoch: 014, Train Acc: 0.6333, Test Acc: 0.5769, Loss: 0.6883
Epoch: 015, Train Acc: 0.6133, Test Acc: 0.4615, Loss: 0.6936
Epoch: 016, Train Acc: 0.6000, Test Acc: 0.4615, Loss: 0.6808
Epoch: 0

In [ ]:
from torch.nn import Linear
from torch_geometric.nn import GINConv, JumpingKnowledge, global_mean_pool
from torch.nn import BatchNorm1d as BN
from torch.nn import Linear, ReLU, Sequential
#from torch_geometric.nn import JumpingKnowledge, SAGEConv, global_mean_pool

class GIN(torch.nn.Module):
    def __init__(self):
        super(GIN, self).__init__()
        self.conv1 = GINConv(
            Sequential(
                Linear(4, 16),
                ReLU(),
                Linear(16, 16),
                ReLU(),
                BN(16),
            ), train_eps=False)
        self.conv2 = GINConv(
            Sequential(
                Linear(16, 32),
                ReLU(),
                Linear(32, 32),
                ReLU(),
                BN(32),
            ), train_eps=False)
        self.conv3 = GINConv(
            Sequential(
                Linear(32, 32),
                ReLU(),
                Linear(32, 32),
                ReLU(),
                BN(32),
            ), train_eps=False)
        self.lin1 = Linear(32, 16)
        self.lin2 = Linear(16, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = self.conv3(x, edge_index)
        x = global_mean_pool(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GIN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 200):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4800, Test Acc: 0.5000, Loss: 0.6481
Epoch: 002, Train Acc: 0.4800, Test Acc: 0.5000, Loss: 0.5833
Epoch: 003, Train Acc: 0.5333, Test Acc: 0.5192, Loss: 0.5144
Epoch: 004, Train Acc: 0.5467, Test Acc: 0.4615, Loss: 0.5801
Epoch: 005, Train Acc: 0.5333, Test Acc: 0.5192, Loss: 0.5267
Epoch: 006, Train Acc: 0.6867, Test Acc: 0.5192, Loss: 0.5314
Epoch: 007, Train Acc: 0.7467, Test Acc: 0.5192, Loss: 0.6229
Epoch: 008, Train Acc: 0.6467, Test Acc: 0.4038, Loss: 0.5232
Epoch: 009, Train Acc: 0.5533, Test Acc: 0.4615, Loss: 0.4937
Epoch: 010, Train Acc: 0.4800, Test Acc: 0.5000, Loss: 0.4808
Epoch: 011, Train Acc: 0.7267, Test Acc: 0.5385, Loss: 0.5356
Epoch: 012, Train Acc: 0.7333, Test Acc: 0.6346, Loss: 0.4684
Epoch: 013, Train Acc: 0.6333, Test Acc: 0.4423, Loss: 0.3092
Epoch: 014, Train Acc: 0.7000, Test Acc: 0.4423, Loss: 0.6177
Epoch: 015, Train Acc: 0.6000, Test Acc: 0.5192, Loss: 0.4684
Epoch: 016, Train Acc: 0.4867, Test Acc: 0.5000, Loss: 0.6587
Epoch: 0

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.data import Batch
from torch_geometric.nn import (
    GraphConv,
    JumpingKnowledge,
    global_mean_pool,
    graclus,
    max_pool,
)

num_layers  = 4
hidden = 16
class Graclus(torch.nn.Module):
    def __init__(self):
        super(Graclus, self).__init__()
        self.conv1 = GraphConv(4, 16, aggr='mean')
        self.convs = torch.nn.ModuleList()
        for i in range(num_layers - 1):
            self.convs.append(GraphConv(16, 16, aggr='mean'))
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(64, 16)
        self.lin2 = Linear(16, 1)

    def forward(self, data):
        x, edge_index, weight , batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                cluster = graclus(edge_index,num_nodes=x.size(0))
                data = Batch(x=x, edge_index=edge_index, batch=batch)
                data = max_pool(cluster, data)
                x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
        return x

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Graclus().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 500):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6877
Epoch: 002, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.7292
Epoch: 003, Train Acc: 0.4800, Test Acc: 0.5000, Loss: 0.6937
Epoch: 004, Train Acc: 0.4800, Test Acc: 0.5000, Loss: 0.6971
Epoch: 005, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6881
Epoch: 006, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.7188
Epoch: 007, Train Acc: 0.5000, Test Acc: 0.5769, Loss: 0.6931
Epoch: 008, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.7303
Epoch: 009, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6876
Epoch: 010, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6998
Epoch: 011, Train Acc: 0.6067, Test Acc: 0.5385, Loss: 0.6905
Epoch: 012, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.7077
Epoch: 013, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.7294
Epoch: 014, Train Acc: 0.5467, Test Acc: 0.5577, Loss: 0.6999
Epoch: 015, Train Acc: 0.5267, Test Acc: 0.5192, Loss: 0.6924
Epoch: 016, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6965
Epoch: 0

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    GraphConv,
    JumpingKnowledge,
    TopKPooling,
    global_mean_pool,
)


class TopK(torch.nn.Module):
    def __init__(self):
        super(TopK, self).__init__()
        self.ratio = 0.8
        self.num_layers = 3
        self.hidden = 16
        self.conv1 = GraphConv(4, 16, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(16, 16, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend(
            [TopKPooling(16, self.ratio) for i in range((self.num_layers) // 2)])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(48, 16)
        self.lin2 = Linear(16, 1)


    def forward(self, data):
        x, edge_index, weight, batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, _, batch, _, _ = pool(x, edge_index,
                                                     batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        #return F.log_softmax(x, dim=-1)
        return x

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TopK().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4800, Test Acc: 0.5000, Loss: 0.7532
Epoch: 002, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6802
Epoch: 003, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.7067
Epoch: 004, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.7109
Epoch: 005, Train Acc: 0.4933, Test Acc: 0.4423, Loss: 0.6939
Epoch: 006, Train Acc: 0.5533, Test Acc: 0.5577, Loss: 0.6929
Epoch: 007, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6874
Epoch: 008, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6930
Epoch: 009, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6984
Epoch: 010, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6842
Epoch: 011, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6921
Epoch: 012, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.7045
Epoch: 013, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6899
Epoch: 014, Train Acc: 0.5267, Test Acc: 0.5192, Loss: 0.6910
Epoch: 015, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6964
Epoch: 016, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6832
Epoch: 0

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    GraphConv,
    JumpingKnowledge,
    SAGPooling,
    global_mean_pool,
)


class SAGPool(torch.nn.Module):
    def __init__(self):
        super(SAGPool, self).__init__()
        self.num_layers = 4
        self.hidden = 32
        self.ratio = 0.8
        self.conv1 = GraphConv(4, self.hidden, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(self.hidden, self.hidden, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend(
            [SAGPooling(self.hidden, self.ratio) for i in range((self.num_layers) // 2)])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(self.num_layers * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, weight, batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, _, batch, _, _ = pool(x, edge_index,
                                                     batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SAGPool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.7232
Epoch: 002, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.7132
Epoch: 003, Train Acc: 0.4800, Test Acc: 0.5000, Loss: 0.7297
Epoch: 004, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.7002
Epoch: 005, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6749
Epoch: 006, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6898
Epoch: 007, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6925
Epoch: 008, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6755
Epoch: 009, Train Acc: 0.4800, Test Acc: 0.5000, Loss: 0.6977
Epoch: 010, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6962
Epoch: 011, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6938
Epoch: 012, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.7175
Epoch: 013, Train Acc: 0.4800, Test Acc: 0.5000, Loss: 0.7000
Epoch: 014, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.7206
Epoch: 015, Train Acc: 0.5267, Test Acc: 0.5000, Loss: 0.6879
Epoch: 016, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6878
Epoch: 0

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    EdgePooling,
    GraphConv,
    JumpingKnowledge,
    global_mean_pool,
)


class EdgePool(torch.nn.Module):
    def __init__(self):
        super(EdgePool, self).__init__()
        self.num_layers = 3 
        self.hidden = 16
        self.conv1 = GraphConv(4, self.hidden, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(self.hidden, self.hidden, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend(
            [EdgePooling(self.hidden) for i in range((self.num_layers) // 2)])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(self.num_layers * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, weight ,  batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, batch, _ = pool(x, edge_index, batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EdgePool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4800, Test Acc: 0.5000, Loss: 0.7576
Epoch: 002, Train Acc: 0.4800, Test Acc: 0.5000, Loss: 0.7412
Epoch: 003, Train Acc: 0.4800, Test Acc: 0.5000, Loss: 0.6937
Epoch: 004, Train Acc: 0.4800, Test Acc: 0.5000, Loss: 0.7067
Epoch: 005, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.7043
Epoch: 006, Train Acc: 0.4800, Test Acc: 0.5000, Loss: 0.7135
Epoch: 007, Train Acc: 0.4800, Test Acc: 0.5000, Loss: 0.7123
Epoch: 008, Train Acc: 0.4800, Test Acc: 0.5000, Loss: 0.6839
Epoch: 009, Train Acc: 0.4800, Test Acc: 0.5000, Loss: 0.6924
Epoch: 010, Train Acc: 0.4800, Test Acc: 0.5000, Loss: 0.6983
Epoch: 011, Train Acc: 0.4800, Test Acc: 0.5000, Loss: 0.6863
Epoch: 012, Train Acc: 0.4800, Test Acc: 0.5000, Loss: 0.6974
Epoch: 013, Train Acc: 0.4800, Test Acc: 0.5000, Loss: 0.6949
Epoch: 014, Train Acc: 0.4800, Test Acc: 0.5000, Loss: 0.6935
Epoch: 015, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6932
Epoch: 016, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6928
Epoch: 0

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import GlobalAttention, SAGEConv


class GlobalAttentionNet(torch.nn.Module):
    def __init__(self):
        super(GlobalAttentionNet, self).__init__()
        self.num_layers = 3
        self.hidden  = 16
        self.conv1 = SAGEConv(4, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(SAGEConv(self.hidden, self.hidden))
        self.att = GlobalAttention(Linear(self.hidden, 1))
        self.lin1 = Linear(self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = self.att(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)


In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GlobalAttentionNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5200, Test Acc: 0.5577, Loss: 0.6948
Epoch: 002, Train Acc: 0.5800, Test Acc: 0.5385, Loss: 0.6862
Epoch: 003, Train Acc: 0.5933, Test Acc: 0.6154, Loss: 0.6863
Epoch: 004, Train Acc: 0.5667, Test Acc: 0.5962, Loss: 0.6791
Epoch: 005, Train Acc: 0.5733, Test Acc: 0.5577, Loss: 0.6937
Epoch: 006, Train Acc: 0.5600, Test Acc: 0.5769, Loss: 0.6950
Epoch: 007, Train Acc: 0.5533, Test Acc: 0.5577, Loss: 0.6906
Epoch: 008, Train Acc: 0.5533, Test Acc: 0.5769, Loss: 0.6940
Epoch: 009, Train Acc: 0.5733, Test Acc: 0.5577, Loss: 0.6741
Epoch: 010, Train Acc: 0.5733, Test Acc: 0.5769, Loss: 0.6881
Epoch: 011, Train Acc: 0.5933, Test Acc: 0.5577, Loss: 0.6952
Epoch: 012, Train Acc: 0.5800, Test Acc: 0.5769, Loss: 0.7036
Epoch: 013, Train Acc: 0.5867, Test Acc: 0.5769, Loss: 0.7012
Epoch: 014, Train Acc: 0.5733, Test Acc: 0.5769, Loss: 0.6960
Epoch: 015, Train Acc: 0.5800, Test Acc: 0.5769, Loss: 0.6833
Epoch: 016, Train Acc: 0.6267, Test Acc: 0.5962, Loss: 0.6791
Epoch: 0

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import SAGEConv, Set2Set


class Set2SetNet(torch.nn.Module):
    def __init__(self):
        super(Set2SetNet,self).__init__()
        self.hidden = 16
        self.num_layers = 3 
        self.conv1 = SAGEConv(4, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(SAGEConv(self.hidden, self.hidden))
        self.set2set = Set2Set(self.hidden, processing_steps=4)
        self.lin1 = Linear(2 * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = self.set2set(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Set2SetNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4800, Test Acc: 0.5000, Loss: 0.6395
Epoch: 002, Train Acc: 0.5867, Test Acc: 0.6154, Loss: 0.6958
Epoch: 003, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.7174
Epoch: 004, Train Acc: 0.5400, Test Acc: 0.5000, Loss: 0.7070
Epoch: 005, Train Acc: 0.5667, Test Acc: 0.6154, Loss: 0.6827
Epoch: 006, Train Acc: 0.6400, Test Acc: 0.5192, Loss: 0.6879
Epoch: 007, Train Acc: 0.6200, Test Acc: 0.4808, Loss: 0.7092
Epoch: 008, Train Acc: 0.5933, Test Acc: 0.6538, Loss: 0.7232
Epoch: 009, Train Acc: 0.6133, Test Acc: 0.5000, Loss: 0.7395
Epoch: 010, Train Acc: 0.5867, Test Acc: 0.5385, Loss: 0.6921
Epoch: 011, Train Acc: 0.6533, Test Acc: 0.4808, Loss: 0.7709
Epoch: 012, Train Acc: 0.6067, Test Acc: 0.6346, Loss: 0.6775
Epoch: 013, Train Acc: 0.6667, Test Acc: 0.5385, Loss: 0.6319
Epoch: 014, Train Acc: 0.6067, Test Acc: 0.5769, Loss: 0.7333
Epoch: 015, Train Acc: 0.5733, Test Acc: 0.5769, Loss: 0.7376
Epoch: 016, Train Acc: 0.6533, Test Acc: 0.5385, Loss: 0.6625
Epoch: 0

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn import Conv1d, Linear

from torch_geometric.nn import SAGEConv, global_sort_pool


class SortPool(torch.nn.Module):
    def __init__(self):
        super(SortPool, self).__init__()
        self.k = 30
        self.num_layers = 3 
        self.hidden = 16
        self.conv1 = SAGEConv(4, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(SAGEConv(self.hidden, self.hidden))
        self.conv1d = Conv1d(self.hidden, 32, 5)
        self.lin1 = Linear(32 * (self.k - 5 + 1), self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = global_sort_pool(x, batch, self.k)
        x = x.view(len(x), self.k, -1).permute(0, 2, 1)
        x = F.relu(self.conv1d(x))
        x = x.view(len(x), -1)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SortPool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5267, Test Acc: 0.5385, Loss: 0.6816
Epoch: 002, Train Acc: 0.5533, Test Acc: 0.5769, Loss: 0.6984
Epoch: 003, Train Acc: 0.5533, Test Acc: 0.5577, Loss: 0.6791
Epoch: 004, Train Acc: 0.5867, Test Acc: 0.5385, Loss: 0.7022
Epoch: 005, Train Acc: 0.5800, Test Acc: 0.5000, Loss: 0.6803
Epoch: 006, Train Acc: 0.5667, Test Acc: 0.4615, Loss: 0.6856
Epoch: 007, Train Acc: 0.5800, Test Acc: 0.4808, Loss: 0.6716
Epoch: 008, Train Acc: 0.5867, Test Acc: 0.5000, Loss: 0.6785
Epoch: 009, Train Acc: 0.6400, Test Acc: 0.4808, Loss: 0.6686
Epoch: 010, Train Acc: 0.7000, Test Acc: 0.5192, Loss: 0.6683
Epoch: 011, Train Acc: 0.6667, Test Acc: 0.4423, Loss: 0.6830
Epoch: 012, Train Acc: 0.6333, Test Acc: 0.4615, Loss: 0.6573
Epoch: 013, Train Acc: 0.6200, Test Acc: 0.4808, Loss: 0.6758
Epoch: 014, Train Acc: 0.6400, Test Acc: 0.5000, Loss: 0.6697
Epoch: 015, Train Acc: 0.6733, Test Acc: 0.5385, Loss: 0.6434
Epoch: 016, Train Acc: 0.6667, Test Acc: 0.5192, Loss: 0.6796
Epoch: 0

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SortPool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ASAP().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.7143
Epoch: 002, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6741
Epoch: 003, Train Acc: 0.4933, Test Acc: 0.5192, Loss: 0.6928
Epoch: 004, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.6961
Epoch: 005, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6935
Epoch: 006, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6924
Epoch: 007, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6866
Epoch: 008, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6836
Epoch: 009, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6951
Epoch: 010, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6991
Epoch: 011, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6911
Epoch: 012, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6964
Epoch: 013, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.7004
Epoch: 014, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6928
Epoch: 015, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6940
Epoch: 016, Train Acc: 0.5200, Test Acc: 0.5000, Loss: 0.6959
Epoch: 0